# バックテスト用

input:期間、所持金、使用するルール<br>
output:期間中の所持金の増減具合、図とシャープレシオの値<br>

In [9]:
import sys
sys.path.append('../')

from scraping.scrape_fx_data import scrape

In [10]:
scrape()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
